Convolutional Sentiment Analysis
====

Tradizionalmente le reti convoluzionali sono usate per analizzare le immagini, i layer di convoluzione di solito sono seguiti da uno o piu linear layer. I layer convoluzionali usano filtri (chiamati anche kernel) che scansionano l'immagine e ne creano un'altra. L'idea intuitiva che sta dietro all'apprendimento delle reti convoluzionali è che lavorano come estrattori di feature. Andandosi a concentrare sulle parti più importanti della nostra immagine.

Come si usano le reti convoluzionali sul testo? Ad esempio un filtro 1x2 può controllare due parole sequenziali, bi-gram. 
L'intuizione è che la presenza di alcuni bi-grams o tri-grams in una frase sono un buon indicatore del risultato finale.

Preparazione dei dati
----

Invece di creare i bi-grams come nel modello FastText lasceremo che sia lo strato di convoluzione a fare questo lavoro.
Il layer di convoluzione si aspetta che la dimensione del batch sia la prima, dobbiamo dire a TorchText di preparare i dati in questo modo andando ad esplicitarlo con il parametro batch_first = True.

In [2]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Creiamo il vocabolario e carichiamo il vettore di word embeddings.

In [3]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

Come prima creiamo gli iteratori.

In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

Costuiamo il modello
----

Andiamo a vedere come costruire una CNSS da usare per il testo. Le immagini sono tipicamente bidimensionali (non consideriamo la dimensione dei colori) mentre il testo viene trasformato in una sequenza di numeri (monodimensionale).

Però sappiamo che il primo passo di quasi tutti i notebook precedentei è stato convertire le parole in word embeddings.
Ecco come possiamo immaginare le parole nella seconda dimensione, ogni parola lungo un asse e gli elementi del vettore lungo l'altra. Analizziamo la rappresentazione a due dimensioni della frase seguente:

<img src="./images/conv.png">

Possiamo usare un filtro di dimensione [n x emb_dim]. Questo coprirà $n$ parole in sequenza
Considera l'ìmmagine qui sotto, con i nostri word vectori rappresentati in verde. Abbiamo 4 parole con una dimensionalità di embedding impostata a 5, creiamo dunque un tensore "immagine" [4x5]

Un filtro che copre due parole alla volta dovrà essere un filtro [2x5], mostrato in giallo.
L'output del filtro, in rosso sarà un singolo numero risultato della convoluzione.

<img src="./images/conv1.png">

il filtro si muove in basso e calcola il prossimo risultato della convoluzione

<img src="./images/conv2.png">

Fino alla fine della frase

Nel nostro caso avremo come risultato un vettore con il numero di elementi pari alla lughezza della frase meno l'altezza del filtro più uno nel nostro caso $4-2+1=3$.


L'esempio mostra come calcolare l'output con un solo filtro. Il nostro modello, avrà molti di questi filtri. L'idea è che ogni filtro si concentrerà su una differente feature da estrarre. 

Nel nostro modello avremo anche differenti dimensioni dei filtri, con dimensione 3,4 e 5 con centinaia di questi. L'intuizione è che guarderemo differenti occorrenze di tri-grams, 4-grams and 5-grams che sono rilevanti per l'analisi del sentiment delle recensioni dei nostri film.

Il posso successivo del modello e usare il pooling (max pooling) sull'ouptut del layer convoluzionale.
Questo è simile a quanto fatto nel modello FastText dove andavamo a calcolare la media di ogni word vector, con la funzione F.avg_pool2d, ora invece di calcolare la media su una dimensione, andremo a prendere il valore massimo.
Qui sotto un esempio grafico.

<img src="./images/max_pooling.png">

L'idea è che il valore massimo è la feature "più importante" per determinare il sentiment di una recensione, che corrisponde al n-gram "più importante" della recensione.

Come facciamo a riconoscere l'n-gram più importante? Fortunatamente non dobbiamo farlo noi!. Tramite la backpropagation, i pesi dei filtri sono cambiati in modo da far risultare certi n-gram più indicativi nella recensione che abbiamo letto e dargli un valore più alto.

Il nostro modello ha 100 filtri con 3 differenti dimensioni, questo significa che si concentrerà su 300 differenti n-grams.
Concateneremo i risultato di questi filtri in un singolo vettore e lo passeremo ad un linear layer per ottenere il risultato.

Possiamo pensare ai pesi dell'ultimo livello come un "soppesatore del risultato" per ognino dei 300 n-grams per ottenere il risultato finale.

Implementazione nel dettaglio
----
Implementeremo i layer convoluzionali con la funzione nn.Conv2d. Il parametro in_channels è il numero dei canali nella nostra immagine nel layer convoluzionale. Le immagini di solito ne hanno 3 (il canale rosso,blu e verde), stiamo usando del testo e dunque avremo un canale soltanto. `out_channels` è il numero dei filtri mentre `kernel_size` è la dimensione dei filtri stessi.
Ogni `kernel_sizes` avrà una dimensione [n x emb_dim] dove $n$ è la dimensione dei n-grams.

In PyTorch, le RNN vogliono la dimensione del batch in imput come seconda dimensione, mentre le CNN volgiono la dimensione del batch come prima, non dobbiamo cambiare niente se abbiamo già impostato `batch_first = True` in campo TEXT.
Possiamo poi passare la frase nel nostra layer di embedding. La seconda dimensione del nostro input è il numero di canali da dare alla funzione `nn.Conv2d`. Un testo tecnicamente non a la dimensione channel, eseguiamo un `unsqueeze` del nostro tensore per crearne una.

Passiamo dunque i tensori lungo i layer convoluzionali e il pooling, usiamo anche la activation function RelU dopo ogni convolutional layer.
Un'altra simpatica feature del pooling layer è che si può lavorare frasi con lunghezze differenti.
La dimensione dell'uscita del layer convoluzionale è solo dipendente dal numero di filtri. Senza il layer max pooling l'ingresso del layer linear dipenderebbe dalla dimensione della frase in input (e non è quello che si vuole).
Una opzione sarebbe quella di tagliare o riempire tutte le frasi per averle tutte uguali, comunque con il layer max pooling siamo sicuri che il linear layer sarà sempre ad una dimensione fissa.

**Nota:** Otteremo una eccezione se la nostra frase sarà più corta del più grande filtro utilizzato. Se questo dovesse succedere dobbiamo usare i token `<pad>` per riempire la frase. Comunque nell' IMDb non ci sono frasi più corte di 5 parole perciò possiamo proseguire tranquilli.

Alla fine eseguiamo un dropout sulla concatenazione dei filtri e diamo il tensore il tensore al linear layer per ottenere il risultato.

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

Currentemente il modello CNN può usare solo 3 differenti dimensioni di filtri, ma possiamo milgiorare il codice del nostro modello e rendelo più generico e prendere ogni numero di filtri.

Possiamo fare questo mettendo tutti i nostri filtri convoluzionali in un `nn.ModuleList`, una funzione di PyTorch per gestire una lista di `nn.Modules`.

Se aggiungessimo semplicemente una lista Pyhton, i moduli nella lista non verrebbero "visti" da PyTorch e questo ci darebbe dei problemi. 

Ora possiamo usare una lista arbitraria di dimensioni di filtri, nella parte di codice che esegue la list comprehension creeremo i convolutional layer per ognuno dei filtri richiesti. 

Nel metodo forward passiamo ogni elemento nella lista convolutional layer e lo applichiamo alla frase in ingresso, al risultato applichiamo il max pool prima di concatenare il risultato e passarlo prima al dropout e poi al linear layer.

In [6]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

Possiamo anche implementare il modello sopra usando dei layers 1-dimensional convolutional, dove la dimensione di embedding è la profondità del filtro e il numero dei token è il parametro width.

In [7]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

andiamo ad instanziare il modello 

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

come sempre fatto andiamo a vedere quanti parametri ha il modello

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,600,801 trainable parameters


Andiamo a caricare il vettore di embedding

In [10]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1386,  0.1180,  0.3534,  ...,  0.1226,  0.5973, -0.1702],
        [-0.0786,  0.0541, -0.0993,  ...,  0.2565, -0.1874, -0.4428],
        [-0.3617,  0.6201,  0.1105,  ...,  0.2994, -0.5920,  1.0949]])

Inizializziamo poi il vettore di embedding unk e pad a zero.

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

Train del modello
---

La fase di train del modello non cambia dai precedenti notebook

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [13]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [14]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [18]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.124 | Train Acc: 95.53%
	 Val. Loss: 0.346 |  Val. Acc: 86.50%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.085 | Train Acc: 97.01%
	 Val. Loss: 0.351 |  Val. Acc: 87.20%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.063 | Train Acc: 98.05%
	 Val. Loss: 0.380 |  Val. Acc: 86.79%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.046 | Train Acc: 98.56%
	 Val. Loss: 0.415 |  Val. Acc: 87.19%
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.033 | Train Acc: 99.00%
	 Val. Loss: 0.437 |  Val. Acc: 87.16%


In [19]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.399 | Test Acc: 84.86%


Input personalizzato
---

**Nota:** Come scritto prima, se la frase di input è più breve del filtro più grande avremo un errore. Per evitare questo la nostra funzione `predict_sentiment` accetta iun parametro `min_len`.

Se la frase in ingresso ha meno token di min_len, andremo a riempire la frase con i tag di padding fino a raggiungere quota min_len.

In [20]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

con una frase negativa

In [21]:
predict_sentiment(model, "This film is terrible")

0.07555589079856873

con una frase positiva

In [23]:
predict_sentiment(model, "This film is great")

0.9691400527954102